### Universidad Nacional de Córdoba - Facultad de Matemática, Astronomía, Física y Computación

### Diplomatura en Ciencia de Datos, Aprendizaje Automático y sus Aplicaciones 2022
Búsqueda y Recomendación para Textos Legales

Mentor: Jorge E. Pérez Villella

# Práctico Introducción al Aprendizaje Automático

Integrantes:
* Fernando Agustin Cardellino
* Adrian Zelaya

### Objetivos:

Probar distintos modelos de clasificación para evaluar la performance y la exactitud de predicción de cada modelo. 

* Utilizando el corpus normalizado en el práctico anterior, transformar el texto en vectores numéricos utilizando scikit-learn comparando los 3 modelos de vectorización. Explicar cada uno estos modelos.

* Clasificar los documentos por fuero. Trabajaremos con los siguientes modelos de clasificación de la librería scikit-learn: Logistic Regresion, Naive Bayes y SVM. En cada modelo probar distintos hiperparámetros, generar la Matriz de Confusión y la Curva ROC. Explicar los resultados obtenidos.

* Determinar y justificar cual es el modelo con mejor performance.

* Predecir el fuero de un documento utilizando el mejor modelo.

Opcional:

* Opcional: Profundizar el tema de stop words y cómo generar uno propio.

Fecha de Entrega: 29 de julio de 2022



In [2]:
# Importamos las librerías necesarias

import numpy as np
import pandas as pd
import os
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from wordcloud import WordCloud, ImageColorGenerator
from nltk.corpus import stopwords
from nltk.stem import porter, snowball
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
#import nltk
#nltk.download('stopwords')  # Descomentar para bajar las stopwords
# Ref: https://machinelearningmastery.com/prepare-text-data-machine-learning-scikit-learn/

In [3]:
# Cargamos modelo de spacy y stopwords de NLTK
nlp = spacy.load("es_core_news_sm")

CURR_DIR = os.getcwd()  # Gets current directory
STOPWORDS_ES = stopwords.words('spanish')
BREAKPOINT=5  # None para analizar todos los documentos, sino un número para analizar hasta n documentos
MAX_WORDS=1000
IMG_NAME = "legal-icon-png"


### Funciones que vamos a utilizar en el Práctico

#### Funciones auxiliares

In [4]:
def getListOfFiles(dirName, return_dir=False):
    # create a list of file and sub directories
    # names in the given directory
    files = os.listdir(dirName)
    allFiles = list()
    # Iterate over all the entries
    for file in files:
        # Create full path
        fullPath = os.path.join(dirName, file)
        # If entry is a directory then get the list of files in this directory
        if os.path.isdir(fullPath):
            if return_dir:
                allFiles.append(fullPath.split(os.sep)[-1])
            else:
                allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)

    return allFiles

#### Funciones específicas del análisis

In [5]:
# Funciones de preprocesamiento

def get_tokens(files_path, fuero_name=None, breakpoint=None, object_type=None):
    """
    Función que arma el corpus de palabras y las tokeniza, utilizando SpaCy.
    Si `breakpoint` != None, dar el número de docs con el que se interrumpe la función
    object_type: indica el tipo de objeto que devuelve, i.e. 'token'/None (SpaCy object `token`), 
    'word' (texto/string) or 'entity' (SpaCy object `entity`) 
    """
    def get_conditions(token, case):
        if case == 'entity':
            return True
        else:
            return token.is_alpha and not token.is_stop
    
    corpus = [] # En esta lista guardamos diccionarios con data de cada documento

    tokens = [] # En esta lista guardamos los tokens de todo el documento

    i = 0
    for filename in getListOfFiles(files_path):
        tokens_ = [] # En esta lista guardamos los tokens por documento
        file_name = filename.split(os.sep)[-1]
        # extraemos el nombre del fuero a partir de la carpeta en donde reside el documento:
        fuero = fuero_name if fuero_name is not None else filename.split(os.sep)[-2]

        with open(filename, encoding='utf-8') as file:
            file_text = file.read()
            # Tokenizamos el corpus
            nlp_doc = nlp(file_text)            
            
            iterable = nlp_doc.ents if object_type == 'entity' else nlp_doc
            
            for token in iterable:
                if get_conditions(token, case=object_type):
                    # lematizamos y pasamos a minúscula (en modo 'palabras'/'word' solamente)
                    aux_token = token.lemma_.lower() if object_type == 'word' else token
                    
                    # insertamos los tokens en la lista de todos los tokens
                    tokens.append(
                        aux_token
                    )
                    # insertamos los tokens en la lista de los tokens específicos al documento
                    tokens_.append(
                        aux_token
                    )
            # insertamos la data del documento a la lista reservada para tal fin
            corpus.append({
                'id': i, 'archivo': file_name, 'fuero': fuero, 'texto': file_text, 'texto_clean': tokens_
            })
        
        i += 1            
        # cortamos la ejecución de acuerdo al valor de quiebre (usado para testeo)
        if breakpoint:
            if i > breakpoint:
                break
    
    return tokens, pd.DataFrame(corpus)


def get_lemmas_stem_from_tokens(tokens, stemmer, language='spanish'):
    aux_dict = {
        'word': [token.lower_ for token in tokens],
        'lemma': [token.lemma_.lower() for token in tokens],
        'stem': [stemmer.stem(token.lower_) for token in tokens]
    }
    return pd.DataFrame(aux_dict)
    
    
def get_conteo_palabras(palabras):
    """Función que genera un pandas DataFrame con la frecuencia de las palabras
    """
    palabras_df = pd.DataFrame([{'palabra': str(x).lower()} for x in palabras])
    # print(corpus_df.head())

    return palabras_df.groupby(['palabra'])['palabra'].count().sort_values(ascending=False)

    
def generar_corpus(file_path, breakpoint, fuero_name=None, verbose=True, object_type='word'):
    if verbose:
        print("Generamos Corpus de palabras y conteo de frecuencias")
    
    # Generamos el corpus de palabras, y el diccionario con el mapeo de los fueros con sus respectivos documentos
    palabras, corpus = get_tokens(file_path, fuero_name=fuero_name, breakpoint=breakpoint, object_type='word')

    frecuencia_palabras_df = get_conteo_palabras(palabras)
    
    if verbose:
        print(f"Algunas palabras: {palabras[:5]}")
    
    return frecuencia_palabras_df, palabras, corpus


def vectorize_corpus(vectorizer, corpus):
    """
    Función que vectoriza un corpus de palabras compuesto por documentos, utilizando cualquier vectorizador de 
    scikit-learn que sea ingresado por el usuario.
    Se asume que el corpus ya se encuentra tokenizado (mediante la función generar_corpus()), por lo que se 'anula'
    el tokenizador por defecto de los vectorizadores.
    """
    vectorizer_ = vectorizer(
        tokenizer=lambda doc: doc, # Pisamos el tokenizador para que tome los tokens como vienen (ver descripción)
        lowercase=False # Paso ya incluido en nuestro preprocesamiento
    )
    return vectorizer_.fit_transform(corpus)

In [6]:
#

In [7]:
# Ubicación de los documentos
filesDir = os.path.join(CURR_DIR, "Documentos")

# Obtenemos lista de los fueros
fueros = getListOfFiles(filesDir, return_dir=True)

In [18]:
n_docs = BREAKPOINT if not None else 'todos'
    
print(f"\nAnálisis para {n_docs} documentos\n")

frecuencia_palabras_df, corpus, corpus_df = generar_corpus(filesDir, BREAKPOINT, object_type='word', verbose=False)

print(frecuencia_palabras_df.head())

print(corpus[:10])

print(corpus_df.head())


Análisis para 5 documentos

palabra
art       80
luis      64
ley       62
vocal     62
doctor    54
Name: palabra, dtype: int64
['sala', 'laboral', 'tribunal', 'superior', 'protocolo', 'sentencias', 'nº', 'resolución', 'año', 'tomo']
   id                                            archivo    fuero  \
0   0                          9 BAEZ-FLECHA BUS.pdf.txt  LABORAL   
1   1                            90 FUNES-COYSPU.pdf.txt  LABORAL   
2   2                     1 QUINTEROS-CONSOLIDAR.pdf.txt  LABORAL   
3   3  3 SANGUEDOLCE-MUNICIPALIDAD DE VILLA ALLENDE.p...  LABORAL   
4   4                        188 LUCIANO-NICOLAS.pdf.txt  LABORAL   

                                               texto  \
0  SALA LABORAL - TRIBUNAL SUPERIOR\n\nProtocolo ...   
1  SALA LABORAL - TRIBUNAL SUPERIOR\n\nProtocolo ...   
2  SALA LABORAL - TRIBUNAL SUPERIOR\n\nProtocolo ...   
3  SALA LABORAL - TRIBUNAL SUPERIOR\n\nProtocolo ...   
4  SALA LABORAL - TRIBUNAL SUPERIOR\n\nProtocolo ...   

            

In [19]:
print(len(corpus))

4920


# Transformación del texto en vectores numéricos

Utilizando el corpus normalizado en el práctico anterior, transformar el texto en vectores numéricos utilizando scikit-learn comparando los 3 modelos de vectorización. Explicar cada uno estos modelos.

## CountVectorizer

Este vectorizador convierte una colección de textos en una matriz de recuentos de tokens.

Para ello, primero **tokeniza** las palabras y asigna un token id (número entero) a cada token. El tokenizador por default utiliza espacios y separadores de puntuación como separadores. Luego, **cuenta** las ocurrencias de los tokens en cada documento.

El producto de salida es una matriz rala/dispersa, donde cada fila representa un documento mientras que cada columna representa un token. Los valores de la matriz representan el número de ocurrencias del token en el documento.

Referencias:
1. [sckit-learn documentation - CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)
2. [sckit-learn user guide - Text feature extraction](https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction) 


In [20]:
# CountVectorizer

X_1 = vectorize_corpus(CountVectorizer, corpus_df['texto_clean'])#corpus)

print(X_1.toarray())
print(X_1.toarray().shape)


[[0 2 0 ... 1 1 0]
 [0 0 1 ... 0 1 0]
 [0 0 0 ... 0 0 1]
 [0 2 0 ... 1 0 0]
 [1 0 1 ... 0 0 0]
 [0 0 3 ... 1 0 0]]
(6, 1406)


## TfidfVectorizer

Este vectorizador convierte una colección de textos en una matriz de coeficientes **TF-IDF** (**T**erm-**F**requency - **I**nverse **D**ocument-**F**requency). Por la naturaleza del método, suele ser más útil en textos largos dado que para textos cortos los vectores resultantes pueden ser ruidosos/inestables. Es equivalente a utilizar el vectorizador `CountVectorizer` seguido del transformador `TfidfTransformer`.

Los coeficientes TF-IDF nos permiten balancear el peso de las palabras de acuerdo a su frecuencia de ocurrencia, dándole menos importancia a palabras que se repiten seguido y resaltando aquellas que son más inusuales/raras.

Para ello se aplica la siguiente transformación:

$\text{tf-idf}_{(t,d)} = \text{tf}_{(t,d)} * \text{idf}_{(t)}$ ,

donde $t$ corresponde a un término (token/palabra) y $d$ a un documento, además:

$\text{tf}_{(t,d)} = \frac{\text{ocurrencias de } t \text{ en } d }{\text{número de palabras en }d}$ $ $ y $ $  $\text{idf}_{(t)} = \log{\frac{1+n}{1+\text{df}_{(t)}}}$ ,

donde $n$ es el número total de documentos en el corpus y $\text{df}$ es el número de documentos que contienen el término/palabra $t$.

Finalmente, los vectores resultantes de cada documento, i.e. fila de la matriz, es normalizado utilizando la norma seleccionada (default: Euclideana o $L^2$):

$v_{norm} = \frac{v}{||v||_2} = \frac{v}{\sqrt{v{_1}^2 + v{_2}^2 + \dots + v{_n}^2}}$

Referencias:
1. [sckit-learn documentation - TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html#sklearn.feature_extraction.text.TfidfVectorizer)
2. [sckit-learn user guide - Text feature extraction](https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction) 

In [15]:
# ver normas L1 o L2 (default: L2)
# ver de pasar los documentos y configurar los parámetros de preprocesamiento
X_2 = vectorize_corpus(TfidfVectorizer, corpus_df['texto_clean'])


print(X_2.toarray())
print(X_2.toarray().shape)

[[0.         0.04349111 0.         ... 0.01835909 0.02174556 0.        ]
 [0.         0.         0.022768   ... 0.         0.02696772 0.        ]
 [0.         0.         0.         ... 0.         0.         0.03487517]
 [0.         0.05075007 0.         ... 0.02142335 0.         0.        ]
 [0.03892785 0.         0.02695024 ... 0.         0.         0.        ]
 [0.         0.         0.06214147 ... 0.02071382 0.         0.        ]]
(6, 1406)


## HashingVectorizer

Este vectorizador convierte una colección de textos en una matriz de ocurrencias de tokens.

Se diferencia de `CountVectorizer` en que utiliza *feature hashing*, i.e. aplica una función de hash a cada token y usa dicho hash como índice en vez de buscar los índices en alguna tabla asociativa.

Esto hace que sea un método comparativamente mucho más rápido y con un uso reducido de memoria, útil para grandes datasets.

En cuanto a sus desventajas, como no guarda registro de las características de los inputs originales, no es posible aplicar la transformación inversa, lo cual implica que no es posible saber qué características (features) pueden ser más relevantes en el modelo. 

Otro inconveniente con el método es que no permite balancear los tokens según su frecuencia de ocurrencia (IDF), aunque esto se puede suplir incluyendo un `TfidfTransformer` en el pipeline.

Referencias:
1. [sckit-learn documentation - HashingVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.HashingVectorizer.html#sklearn.feature_extraction.text.HashingVectorizer)
2. [sckit-learn user guide - Text feature extraction](https://scikit-learn.org/stable/modules/feature_extraction.html#vectorizing-a-large-text-corpus-with-the-hashing-trick) 
3. [Wikipedia - Feature hashing](https://en.wikipedia.org/wiki/Feature_hashing)

In [14]:
# además de norm, ver n_features, alternate_sign
# ver de pasar los documentos y configurar los parámetros de preprocesamiento
X_3 = vectorize_corpus(HashingVectorizer, corpus_df['texto_clean'])
print(X_3)

  (0, 932)	-0.10306601058361442
  (0, 6495)	0.0343553368612048
  (0, 9176)	0.0171776684306024
  (0, 9520)	0.0171776684306024
  (0, 13227)	-0.0171776684306024
  (0, 16547)	-0.0171776684306024
  (0, 18775)	0.0171776684306024
  (0, 20797)	-0.0171776684306024
  (0, 21666)	0.0343553368612048
  (0, 23868)	-0.0171776684306024
  (0, 24958)	0.0171776684306024
  (0, 25133)	-0.0171776684306024
  (0, 25678)	-0.0171776684306024
  (0, 26115)	-0.0171776684306024
  (0, 26798)	-0.0171776684306024
  (0, 30298)	-0.0171776684306024
  (0, 33190)	0.08588834215301201
  (0, 48516)	-0.15459901587542163
  (0, 49032)	-0.0171776684306024
  (0, 50569)	-0.0171776684306024
  (0, 52575)	-0.05153300529180721
  (0, 54040)	-0.05153300529180721
  (0, 54733)	-0.0171776684306024
  (0, 57213)	-0.0171776684306024
  (0, 58260)	0.0343553368612048
  :	:
  (5, 988158)	0.018847805889708434
  (5, 989406)	0.09423902944854216
  (5, 993559)	0.0565434176691253
  (5, 999479)	0.09423902944854216
  (5, 1006857)	0.018847805889708434
  (5,

/home/adrian/PycharmProjects/Busqueda-y-Recomendacion-para-Textos-Legales-Mentoria-2022/venv/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:524: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


# Clasificación de documentos por fuero

## Regresión logística

In [36]:
### REVISAR !!!!!!
# Logistic Regression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

def simple_logistic_classify(X_tr, y_tr, X_test, y_test, description, _C=1.0):
    model = LogisticRegression(C=_C).fit(X_tr, y_tr)
    score = model.score(X_test, y_test)
    print('Test Score with', description, 'features', score)
    return model


X = corpus_df[[x for x in corpus_df.columns if x != 'fuero']]
y = corpus_df['fuero']


X_train, y_train, X_test, y_test = train_test_split(X, y, test_size=0.3)




    

,id,archivo,texto,texto_clean
4,4,188 LUCIANO-NICOLAS.pdf.txt,SALA LABORAL - TRIBUNAL SUPERIOR\n\nProtocolo ...,"[sala, laboral, tribunal, superior, protocolo,..."
0,0,9 BAEZ-FLECHA BUS.pdf.txt,SALA LABORAL - TRIBUNAL SUPERIOR\n\nProtocolo ...,"[sala, laboral, tribunal, superior, protocolo,..."
5,5,220 MURUA-PROVINCIA.pdf.txt,SALA LABORAL - TRIBUNAL SUPERIOR\n\nProtocolo ...,"[sala, laboral, tribunal, superior, protocolo,..."
3,3,3 SANGUEDOLCE-MUNICIPALIDAD DE VILLA ALLENDE.p...,SALA LABORAL - TRIBUNAL SUPERIOR\n\nProtocolo ...,"[sala, laboral, tribunal, superior, protocolo,..."
